## Convert PDB file generated by RASPA2 to EXTXYZ format

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from pymatgen.io.cif import CifParser
import pickle
plt.rcParams['font.size'] = '15'

In [2]:
# define some parameters
# Constant
k2kjmol = 0.00831446 # constants. Convert K to kJ/mol

# number of movie frames
# there are total of 10k frames available
nframe = 10000

# atom type in adsorbate molecules
ads_atom = ['Ar']

# atom type in host solid adsorbent
host_atom_type = ['Mg','O','C','H']

# MOF information
mof_file = "MgMOF-74-1x1x1-P1-VASP_charged.cif"
sc_fac=  [1,1,2] # multiply by this vector to get a supercell

# Movie file
movie_file = "Movies/System_0/Movie_MgMOF-74-1x1x1-P1-VASP_charged_1.1.2_80000.000000_0.000000_allcomponents.pdb"

In [3]:
# load energy (label) data using numpy and omit header
#h2o_coords = np.loadtxt('DFT_Datasets/H2O/H2O_Coordinates.dat', skiprows=1)
energy = np.loadtxt("host-ads_energies.txt")
energy = energy * k2kjmol # convert to kJ/mol

In [4]:
# Initialize a CifParser object
parser = CifParser(mof_file)

# Parse the CIF file and obtain a Structure object
structure = parser.get_structures(primitive=False)[0]

supercell = structure * sc_fac

In [5]:
# print lattice matrix of supercell 
print(supercell.lattice)

# print the flatterned lattice matrix of the supercell
lattice_extxyz = supercell.lattice.matrix.flatten()
lattice_extxyz

26.136000 0.000000 0.000000
-13.068000 22.634440 0.000000
0.000000 0.000000 13.884000


array([ 2.61360000e+01,  0.00000000e+00,  1.60036844e-15, -1.30680000e+01,
        2.26344400e+01,  1.60036844e-15,  0.00000000e+00,  0.00000000e+00,
        1.38840000e+01])

In [6]:
# Read the movie file into a numpy array
# read PDF file
f = open(movie_file)

# define a structure dtype numpy array
dt = np.dtype([('atom', np.str_, 20), 
               ('coords', np.float64, (3,))]) # change dtype to include only 3 coordinates

# initialize a numpy array to store adsorbate coordinates
ads_cart_coords = np.zeros((nframe,), dtype=dt) # remove the second dimension

# loop over each frame
for i in range(nframe):

    # read first two lines
    line = [next(f).strip().split() for x in range(2)]

    while 1:
        line = next(f).strip().split()

        if line[0] == 'ENDMDL':
            break

        if line[2] in ads_atom:
            
            # read in coordinates of atoms in an adsorbate molecule in cartesian coordinates
            ads_cart_coords[i]['atom'] = line[2]
            ads_cart_coords[i]['coords'] = np.array([float(line[4]), float(line[5]), float(line[6])])


In [7]:
ads_cart_coords[0]

('Ar', [15.105,  3.805, 11.423])

In [8]:
# write both coordinates of adsorbate molecules and framework to extxyz file
# open file for writing
f = open('Ar_MgMOF74.xyz', 'w')

for i in range(nframe):

    # write header
    # total number of atoms in the supercell
    natom = len(supercell) + 1
    f.write(str(natom) + '\n')

    # write comment line
    lattice_string = ("Lattice=" + "\"" + 
                        str(lattice_extxyz[0]) + " " + 
                        str(lattice_extxyz[1]) + " " + 
                        str(lattice_extxyz[2]) + " " +
                        str(lattice_extxyz[3]) + " " +
                        str(lattice_extxyz[4]) + " " +
                        str(lattice_extxyz[5]) + " " +
                        str(lattice_extxyz[6]) + " " +
                        str(lattice_extxyz[7]) + " " +
                        str(lattice_extxyz[8]) + "\"" + 
                        " " + "Properties=species:S:1:pos:R:3" + " " +
                        "energy=" + str(energy[i]) + " " +
                        "pbc=\"T T T\"" + '\n')
    f.write(lattice_string)

    # write adsorbate molecule coordinates first
    ads_string = ( str(ads_cart_coords[i]['atom']) + "       " +
                    str(ads_cart_coords[i]['coords'][0]) + "       " + 
                    str(ads_cart_coords[i]['coords'][1]) + "       " + 
                    str(ads_cart_coords[i]['coords'][2]) + '\n' )
    
    f.write(ads_string)

    # write framework coordinates
    for jhatom in range(len(supercell)):
        atom = supercell[jhatom].species_string
        coords = supercell[jhatom].coords
        host_string = ( str(atom) + "       " + 
                          str(coords[0]) + "       " + 
                          str(coords[1]) + "       " + 
                          str(coords[2]) + '\n' )
        f.write(host_string)
    
# close file
f.close()


    

In [44]:
# # read extxyz file using ase
# import ase
# from ase.visualize import view
# file = ase.io.read('Ar_MgMOF74.extxyz', index=':')